In [1]:
import numpy as np
import pandas as pd
import datetime
import os

from openai import OpenAI

API_PATH = "../config/openai_api.key"

def load_openai_key(path) -> str:
    if not os.path.exists(path):
        raise FileNotFoundError(f"❌ API key file not found: {path}")
    with open(path, "r", encoding="utf-8") as f:
        key = f.read().strip()
    if not key.startswith("sk-"):
        raise ValueError("⚠️ Invalid OpenAI API key format.")
    os.environ["OPENAI_API_KEY"] = key   # ✅ 반드시 환경변수 등록
    print("🔑 OPENAI_API_KEY set.")
    return key

In [2]:
api_key = load_openai_key(API_PATH)

🔑 OPENAI_API_KEY set.


In [3]:
import os
import json
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

class MacroRetriever :
    """
    RAG 기반 Macro Retriever:
    FRED, 경제뉴스, 리서치 리포트 등 외부 데이터를 벡터화해 저장/검색한다.
    """
    def __init__(self, vector_path : str = 'data/vectorstore.faiss') :
        self.vector_path = vector_path
        load_openai_key(API_PATH)
        self.embeddings = OpenAIEmbeddings(model = "text-embedding-3-large")
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100
        )
        self.vectorstore = None

    def load_documents(self, urls : list[str]):
        """
        주어진 URL 리스트에서 문서를 로드하여 Document 객체 리스트로 반환
        """
        all_docs = []
        for url in urls :
            try :
                print(f"🌐 Loading: {url}")
                loader = WebBaseLoader(url)
                docs = loader.load()
                all_docs.extend(docs)
            except Exception as e:
                print(f"⚠️ Failed to load {url}: {e}")
        print(f"✅ Loaded {len(all_docs)} documents.")
        return all_docs

    def build_vectorstore(self, docs):
        """
        로드한 문서를 청크화하고 벡터스토어로 변환.
        """
        print("🔍 Splitting and embedding documents...")
        texts = self.text_splitter.split_documents(docs)
        vectorstore = FAISS.from_documents(texts, self.embeddings)
        print("✅ Vectorstore created.")
        self.vectorstore = vectorstore
        return vectorstore

    def save_vectorstore(self):
        """
        벡터스토어를 로컬 디스크에 저장.
        """
        if self.vectorstore is None:
            raise ValueError("⚠️ No vectorstore to save.")
        os.makedirs(os.path.dirname(self.vector_path), exist_ok=True)
        self.vectorstore.save_local(self.vector_path)
        print(f"💾 Saved vectorstore to {self.vector_path}")

    def load_vectorstore(self):
        """
        기존 벡터스토어를 로드.
        """
        if not os.path.exists(self.vector_path):
            raise FileNotFoundError("No vectorstore found.")
        self.vectorstore = FAISS.load_local(self.vector_path, self.embeddings, allow_dangerous_deserialization=True)
        print("✅ Loaded existing vectorstore.")
        return self.vectorstore

    def retrieve(self, query: str, k: int = 5):
        """
        벡터스토어에서 질의(query)와 가장 유사한 문서 k개 검색.
        """
        if self.vectorstore is None:
            self.load_vectorstore()
        retriever = self.vectorstore.as_retriever(search_kwargs={"k": k})
        results = retriever.get_relevant_documents(query)
        print(f"🔎 Retrieved {len(results)} documents for query: '{query}'")
        return results

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
urls = [
    "https://tradingeconomics.com/united-states/inflation-cpi",
    "https://tradingeconomics.com/united-states/interest-rate",
]

retriever = MacroRetriever()
docs = retriever.load_documents(urls)
vectorstore = retriever.build_vectorstore(docs)
retriever.save_vectorstore()

🔑 OPENAI_API_KEY set.


/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_8837/196366641.py:16: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  self.embeddings = OpenAIEmbeddings(model = "text-embedding-3-large")


🌐 Loading: https://tradingeconomics.com/united-states/inflation-cpi
🌐 Loading: https://tradingeconomics.com/united-states/interest-rate
✅ Loaded 2 documents.
🔍 Splitting and embedding documents...
✅ Vectorstore created.
💾 Saved vectorstore to data/vectorstore.faiss


In [5]:
vectorstore

In [6]:
retriever

In [7]:
retriever.load_vectorstore()
results = retriever.retrieve("What is the current inflation outlook in the U.S.?")
for r in results:
    print(r.page_content[:300], "\n---")

✅ Loaded existing vectorstore.


/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_8837/196366641.py:77: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


🔎 Retrieved 5 documents for query: 'What is the current inflation outlook in the U.S.?'
Inflation Rate in the United States increased to 3 percent in September from 2.90 percent in August of 2025. Inflation Rate in the United States is expected to be 3.10 percent by the end of this quarter, according to Trading Economics global macro models and analysts expectations. In the long-term,  
---
3.20
3.40

                                percent
Oct 2025




                                    3-Year Consumer Inflation Expectations
                                

3.00
3.00

                                percent
Oct 2025




                                    5-Year Consumer Inflat 
---
The annual inflation rate in the US rose to 3% in September 2025, the highest since January, from 2.9% in August and below forecasts of 3.1%. Energy prices rose 2.8% on the year, the most since May 2024, after a 0.2% gain in August, led by fuel oil (4.1% vs -0.5%) and gasoline (-0.5% vs -6.6%) while 
---